In [1]:
%load_ext autoreload
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

## テキスト生成
- promptに続く文章が生成される

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

# モデル名（GPT-2 日本語版）
model_name = "colorfulscoop/gpt2-small-ja"
#model_name = "rinna/japanese-gpt2-medium"

# デバイス設定（GPUがあればCUDA、なければCPU）
device = "cuda" if torch.cuda.is_available() else "cpu"

# モデルとトークナイザーのロード
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


# 文章生成関数
def generate_sentence(prompt="犬が", max_length=50):
    """プロンプトにフォーマットを適用して短い日本語文を生成"""
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=50,  # 50トークンまで生成
            min_length=20,  # 最低20トークン以上生成
            do_sample=True,  # ランダムサンプリングを使用
            temperature=0.7,  # 多様性を持たせる
            top_p=0.9,  # 高確率の単語を優先
            top_k=40,  # 40個の候補から選択
            repetition_penalty=1.2,  # 同じ単語の繰り返しを抑制
            no_repeat_ngram_size=2,  # 2単語以上の繰り返しを防止
            length_penalty=1.0,  # 文章の長さを適切に維持
            num_beams=3,  # ビームサーチの分岐数
            early_stopping=True,  # 途中で適切なタイミングで終了
            num_return_sequences=1,  # 1つの文章を出力
            pad_token_id=tokenizer.eos_token_id  # 文末トークンを設定
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)

# 実行（プロンプトを指定可能）
prompt = "猫が"
text = generate_sentence(prompt)
#text = text.split("。")[0]
print(text)

c:\Users\yutou\Desktop\work\18_LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


猫が大好物で、よく食べているという。また、お酒が飲めるのも好きである。好きな色はピンク。嫌いな食べ物はトマト。苦手な物はカボチャ、ニンニク、タマネギ、


## 文章から単語を取得

- 単語の取得
- 文章の○○置換

In [47]:
from janome.tokenizer import Tokenizer

# Janomeの形態素解析器を初期化
tokenizer = Tokenizer()

# テスト用の文章
text = "今日は天気が良いので、公園で散歩をしました。"

# 解析
for token in tokenizer.tokenize(text):
    n = "".join(token.part_of_speech.split(",")[:2][::-1])
    if n == "一般名詞":
        print(f"{token.surface} - 品詞: {token.part_of_speech}", n)

天気 - 品詞: 名詞,一般,*,* 一般名詞
公園 - 品詞: 名詞,一般,*,* 一般名詞


'句点記号'

In [ ]:
def replace_Noun_something(text):
    

In [9]:
import glob
import pandas as pd
from tqdm import tqdm
import random

paths = glob.glob("C://Program Files (x86)/MeCab/dic/ipadic/Noun.csv")
df = pd.concat([pd.read_csv(path, encoding="shift-jis", header=None) for path in tqdm(paths)]).reset_index(drop=True).reset_index()
df = df.iloc[:, [0, 1, 5, 6, 12]]
df.columns = ["id", "word", "type", "type_detail", "kana"]

100%|██████████| 1/1 [00:00<00:00,  6.81it/s]


In [14]:
for word in random.sample(df["word"].tolist(), 100):
    #print(word)
    text = generate_sentence(word).split("。")[0]
    print(word, text)
    break

ドナーカード ドナーカードを提示すると、そのカードに書かれている内容が表示される
